In [3]:
using QuantumOptics
using PyPlot
PyPlot.pygui(true)
using PyCall
pyyetiPSD = pyimport("pyyeti.psd")
using XLSX: XLSX
np = pyimport("numpy")
using Statistics

In [4]:
freq1 = []
psd1 = []
XLSX.openxlsx("test1.xlsx", enable_cache = false) do f
	sheet = f["Sheet1"]
	for r in XLSX.eachrow(sheet)
		# r is a `SheetRow`, values are read  
		# using column references 
		append!(freq1, r[1])    # will read value at column 1 
		append!(psd1, r[4])    # will read value at column 2 
	end
end

spec1 = np.array([freq1, psd1])
spec1 = np.transpose(spec1)

# https://pyyeti.readthedocs.io/en/latest/modules/generated/pyyeti.psd.psd2time.html
# Everything is in seconds/Hz
sig1, sr1, time1 = pyyetiPSD.psd2time(spec1, fstart = 10^5, fstop = 1.5 * 10^7, ppc = 30, df = 50000, gettime = true)


freq2 = []
psd2 = []
XLSX.openxlsx("test2.xlsx", enable_cache = false) do f
	sheet = f["Sheet1"]
	for r in XLSX.eachrow(sheet)
		# r is a `SheetRow`, values are read  
		# using column references 
		append!(freq2, r[1])    # will read value at column 1 
		append!(psd2, r[4])    # will read value at column 2 
	end
end

spec2 = np.array([freq2, psd2])
spec2 = np.transpose(spec2)

# https://pyyeti.readthedocs.io/en/latest/modules/generated/pyyeti.psd.psd2time.html
# Everything is in seconds/Hz
sig2, sr2, time2 = pyyetiPSD.psd2time(spec2, fstart = 10^5, fstop = 1.5 * 10^7, ppc = 30, df = 50000, gettime = true)

if (time1 != time2)
    println("Time vectors are not the same")
end

plot(time1, sig1)
plot(time2, sig2)
PyPlot.show(block = false)

In [5]:
# Parameters
γ2 = 0.2 # in MHz, intermediate state decay to |1>
γ3 = 0.01 # in MHz, Rydberg decay to |4> (some other states we don't care about). 0.01MHz ~ 100us
Ω1 = Ω2 = 5     # in MHz 
Δ = 0     # in MHz
tmax = 8  # in μs
dt = 0.001
# tlist = [0:dt:tmax;]
tlist = time1*10^6 # time list that we'll evolve our Hamiltonian in μs

9000-element Vector{Float64}:
  0.0
  0.0022222222222222222
  0.0044444444444444444
  0.006666666666666667
  0.008888888888888889
  0.011111111111111112
  0.013333333333333334
  0.015555555555555557
  0.017777777777777778
  0.02
  ⋮
 19.979999999999997
 19.98222222222222
 19.984444444444442
 19.986666666666665
 19.988888888888887
 19.99111111111111
 19.993333333333332
 19.995555555555555
 19.997777777777777

In [6]:
# Basis and operators
b = NLevelBasis(4)
σ1 = transition(b, 1, 2)  # |1><2|
σ2 = transition(b, 2, 3)  # |2><3|
Ryd = transition(b, 4, 3)  # |4><3|, decay out of the system
proj1 = transition(b, 1, 1) # |1><1|
proj2 = transition(b, 2, 2) # |2><2|
proj3 = transition(b, 3, 3)     # |3><3|

Operator(dim=4x4)
  basis: NLevel(N=4)sparse([3], [3], ComplexF64[1.0 + 0.0im], 4, 4)

In [7]:
# Hamiltonian and jump operators
J = [γ2 * σ1 + γ3 * Ryd];
Jdagger = [γ2 * dagger(σ1) + γ3 * dagger(Ryd)];

# Initial state
ψ0 = nlevelstate(b, 1)


Ket(dim=4)
  basis: NLevel(N=4)
 1.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im

In [8]:
# Define the Gaussian function
function gaussian(t, mu, sigma, A)
    return A * exp(-(t - mu)^2 / (2 * sigma^2))
end

# Phase fluctuation for the first driving term
function phi1(t)
    index = round(Int, t / 10^6 * sr1) + 1
    return sig1[index]
end

# Phase fluctuation for the second driving term, but for now it's the same as phi1
function phi2(t)
    index = round(Int, t / 10^6 * sr2) + 1
    return sig2[index]
end


mu1 = 4.1  # center of the peak in us
mu2 = 3  # center of the peak in us
sigma = 0.8 # span of in us
tau = 0.01  # time resolution of the pulse in us

white_noise = 0.1

# Define the driving term functions
function H(t, psi)
    # Round the current time to the nearest multiple of tau
    t_rounded = round(t / tau) * tau
    return 0.5*(σ1*exp(1im*phi1(t))+dagger(σ1)*exp(-1im*phi1(t)))*gaussian(t_rounded, mu1, sigma, Ω1)+0.5*(σ2*exp(1im*phi2(t))+dagger(σ2)*exp(-1im*phi2(t)))*gaussian(t_rounded, mu2, sigma, Ω2)+Δ*proj2+1im*γ3*proj3, J, Jdagger
end

# h_mu2 = H(mu2,ψ0)

# Plots
figure(figsize=(6, 3))
plot(tlist, gaussian.(tlist, mu1, sigma, Ω1), label="1-2")
plot(tlist, gaussian.(tlist, mu2, sigma, Ω2), label="2-3")
axis([0, tmax, 0, Ω1])
legend()
PyPlot.show(block=false)

#=
function H_pump2(t, psi)
    mu = 3e-6  # center of the peak at 3us
    sigma = 1e-6  # span of 1us
    tau = 1e-9  # time resolution of 1ns
    # Round the current time to the nearest multiple of tau
    t_rounded = round(t / tau) * tau
    return dagger(σ2)*gaussian(t_rounded, mu, sigma, Ω2)
end
=#

In [9]:
# Expectation values
function calc_pops(t, ρ)
	p1 = real(expect(proj1, ρ))
	p2 = real(expect(proj2, ρ))
	p3 = real(expect(proj3, ρ))
	return p1, p2, p3
end


calc_pops (generic function with 1 method)

In [10]:
tout = []
pops = []
p1 = []
p2 = []
p3 = []

instance = 100

for i in 1:instance
    # renew the phase fluctuation
    sig1, sr1, time1 = pyyetiPSD.psd2time(spec1, fstart = 10^5, fstop = 1.5 * 10^7, ppc = 30, df = 50000, gettime = true)
    sig2, sr2, time2 = pyyetiPSD.psd2time(spec2, fstart = 10^5, fstop = 1.5 * 10^7, ppc = 30, df = 50000, gettime = true)

    # Time evolution
    toutTemp, popsTemp = timeevolution.master_dynamic(tlist, ψ0, H; fout = calc_pops)

    # Extract populations
    p1Temp = [p[1] for p ∈ popsTemp]
    p2Temp = [p[2] for p ∈ popsTemp]
    p3Temp = [p[3] for p ∈ popsTemp]

    # Store the results
    push!(tout, toutTemp)
    push!(pops, popsTemp)
    push!(p1, p1Temp)
    push!(p2, p2Temp)
    push!(p3, p3Temp)
end


In [11]:
toutAvg = tout[1]
# Reshape pops
p1Avg = mean(p1, dims = 1)[1]
p2Avg = mean(p2, dims = 1)[1]
p3Avg = mean(p3, dims = 1)[1]

p3AvgFinal = round(p3Avg[end], digits = 4)
p3AvgFinalStr = string(p3AvgFinal)

# Plots
figure(figsize = (6, 3))
plot(toutAvg, p1Avg, label = "Initial ground state")
plot(toutAvg, p2Avg, "--", label = "Intermediate state")
plot(toutAvg, p3Avg, label = "Excited state")
title("Final p3: $p3AvgFinalStr")
axis([0, tmax, 0, 1])
legend()
PyPlot.show(block = false)
